In [1]:
import pandas as pd
import numpy as np

import string
import re

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

from tkinter import *
from tkinter import messagebox
from tkinter import filedialog
            

In [2]:
stp=stopwords.words('english')
stp.remove('not')
stp.remove("doesn't")
stp.remove("don't")
stp.remove("didn't")
stp.remove("haven't")
stp.remove("hasn't")
stp.remove("hadn't")
stp.remove("wasn't")
stp.remove("weren't")

df=pd.read_csv('C:/Users/ASUS/ML/dataset/sentiment/Restaurant_Reviews.txt',delimiter='\t')

def remove_punc(msg):
    return re.sub(f'[{string.punctuation}]','',msg)   
def remove_stopwords(msg):
    words=word_tokenize(msg)
    new_words=[]
    for w in words:
        if(w not in stp):
            new_words.append(w)
    return " ".join(new_words) 
ps=PorterStemmer()
def remove_stem(msg):
    words=word_tokenize(msg)
    new_words=[]
    for w in words:
        new_words.append(ps.stem(w))
    return " ".join(new_words) 
def clean_txt(msg):
    X1=remove_punc(msg)
    X2=X1.lower()
    X3=remove_stopwords(X2)
    X4=remove_stem(X3)
    return X4

cv=CountVectorizer(binary=True,ngram_range=(1,2))
X_train=cv.fit_transform(df.Review)
clf=BernoulliNB()
clf.fit(X_train,df.Liked)

def single_prediction(user_input,lbl_result):
    sample=user_input.get()
    X_sample=clean_txt(sample)
    test_sample=cv.transform([X_sample]).toarray()
    pred=clf.predict(test_sample)
    if pred[0]==0:
        lbl_result.configure(text='Not liked',fg='indianred',font=('',30,'bold'))
    else:
        lbl_result.configure(text='Liked',fg='lightgreen',font=('',30,'bold'))
        
def bulk_prediction(entry_src,entry_dest):
    source=entry_src.get()
    destination=entry_dest.get()
    
    dframe=pd.read_csv(source,names=['review'])
    bulk_data=list(map(clean_txt,dframe.review))
    sample=cv.transform(bulk_data).toarray()
    final_data=clf.predict(sample)
    
    dataframe=pd.DataFrame()
    dataframe['review']=dframe.review
    dataframe['sentiment']=final_data
    dataframe['sentiment']=dataframe['sentiment'].map({0:'Not liked',1:'Liked'})
    dataframe.to_csv(destination,index=False,sep='\t')
    messagebox.showinfo('Save','File is Predicted and saved')
    



win=Tk()
win.state('zoom')
win.resizable(width=False,height=True)
win.title('Review Analysis')
win.configure(bg='darkgrey')
lbl_title=Label(win,text='Review Analysis',font=('',50,'bold'),bg='Darkgrey')
lbl_title.place(relx=0.35,rely=0.02)

def home_screen():
    frm=Frame(win,bg='purple')
    frm.place(relx=0,rely=0.15,relwidth=1,relheight=1)
    
    lbl_user=Label(frm,text='Username',font=('',40,'bold'),bg='purple')
    lbl_user.place(relx=0.3,rely=0.3)
    
    entry_user=Entry(frm,font=('',25,'bold'))
    entry_user.place(relx=0.55,rely=0.32)
    entry_user.focus()
    
    lbl_pass=Label(frm,text='Password',font=('',40,'bold'),bg='purple')
    lbl_pass.place(relx=0.3,rely=0.43)
    
    entry_pass=Entry(frm,font=('',25,'bold'),show='*')
    entry_pass.place(relx=0.55,rely=0.45)
    entry_pass.focus()
    
    btn_login=Button(frm,command=lambda:welcome_screen(entry_user,entry_pass),text=' Login ',font=('',25,'bold'))
    btn_login.place(relx=0.63,rely=0.57)

def welcome_screen(entry_user=None,entry_pass=None):
    if (entry_user!= None and entry_pass!= None):
        user=entry_user.get()
        pwd=entry_pass.get()
    else:
        user='admin'
        pwd='password'
    
    if(len(user)==0 or len(pwd)==0):
        messagebox.showerror('Authentication Error','Username/Password is empty')
    else:
        if(user=='admin' and pwd=='password'):
            frm=Frame(win,bg='purple')
            frm.place(relx=0,rely=0.15,relwidth=1,relheight=1)
            
            btn_user=Button(frm,command=lambda:single_feedback(),text='Single feedback prediction',font=('',30,'bold'))
            btn_user.place(relx=0.3,rely=0.2)
            
            btn_user=Button(frm,command=lambda:bulk_feedback(),text=' Bulk feedback prediction ',font=('',30,'bold'))
            btn_user.place(relx=0.305,rely=0.45)
            
            btn_back=Button(frm,command=lambda:confirmation(),text='<- Back ',font=('',18,'bold'))
            btn_back.place(relx=0.02,rely=0.05)
        else:
            messagebox.showerror('Authentication Error','Username/Password is Invalid')

def single_feedback():
    
    username='admin'
    password='password'
    
    frm=Frame(win,bg='purple')
    frm.place(relx=0,rely=0.15,relwidth=1,relheight=1)
    
    lbl_single=Label(frm,text='Enter feedback :',font=('',30,'bold'),bg='purple')
    lbl_single.place(relx=0.2,rely=0.3)
    
    entry_single=Entry(frm,font=('',25,'bold'))
    entry_single.place(relx=0.48,rely=0.315)
    entry_single.focus()
    
    lbl_single=Label(frm,text='Prediction :',font=('',30,'bold'),bg='purple')
    lbl_single.place(relx=0.2,rely=0.58)
    
    lbl_result=Label(frm,text='',font=('',30,'bold'),bg='purple')
    lbl_result.place(relx=0.4,rely=0.58)
    
    btn_single=Button(frm,command=lambda:single_prediction(entry_single,lbl_result),text='Predict',font=('',23,'bold'))
    btn_single.place(relx=0.55,rely=0.43)
    
    btn_back=Button(frm,command=lambda:welcome_screen(),text='<- Back ',font=('',18,'bold'))
    btn_back.place(relx=0.02,rely=0.05)
    
    btn_logout=Button(frm,command=lambda:confirmation(),text='Logout ',font=('',18,'bold'))
    btn_logout.place(relx=0.90,rely=0.05)

    
def bulk_feedback():
    
    username='admin'
    password='password'
    
    frm=Frame(win,bg='purple')
    frm.place(relx=0,rely=0.15,relwidth=1,relheight=1)
    
    lbl_src=Label(frm,text='Import file :',font=('',30,'bold'),bg='purple')
    lbl_src.place(relx=0.2,rely=0.3)
    
    entry_src=Entry(frm,font=('',25,'bold'))
    entry_src.place(relx=0.48,rely=0.315)
    entry_src.focus()
    
    lbl_dest=Label(frm,text='Export prediction :',font=('',30,'bold'),bg='purple')
    lbl_dest.place(relx=0.2,rely=0.45)
    
    entry_dest=Entry(frm,font=('',25,'bold'))
    entry_dest.place(relx=0.48,rely=0.45)
    entry_dest.focus()

    
    btn_src=Button(frm,command=lambda:src_add(entry_src),text='Browse',font=('',18,'bold'))
    btn_src.place(relx=0.72,rely=0.3118)
    
    
    btn_dest=Button(frm,command=lambda:dest_add(entry_dest),text='Browse',font=('',18,'bold'))
    btn_dest.place(relx=0.72,rely=0.446)
    
    btn_result=Button(frm,command=lambda:bulk_prediction(entry_src,entry_dest),text='Predict & Save',font=('',18,'bold'))
    btn_result.place(relx=0.52,rely=0.62)
    
    
    btn_back=Button(frm,command=lambda:welcome_screen(),text='<- Back ',font=('',18,'bold'))
    btn_back.place(relx=0.02,rely=0.05)
    
    btn_logout=Button(frm,command=lambda:confirmation(),text='Logout ',font=('',18,'bold'))
    btn_logout.place(relx=0.90,rely=0.05)

    
def src_add(entry_src):
    path=filedialog.askopenfilename()
    entry_src.delete(0,END)
    entry_src.insert(0,path)
def dest_add(entry_dest):
    path=filedialog.askdirectory()+"/result.txt"
    entry_dest.delete(0,END)
    entry_dest.insert(0,path)

            
def confirmation():
    temp=messagebox.askyesno('Validation','Goto login page')
    if temp==True:
        home_screen()
    else:
        pass
    
    
home_screen()    
win.mainloop()